In [ ]:
#@title Install Packages

In [ ]:
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q nltk
!pip install -q contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [ ]:
!sudo apt-get update
! sudo apt-get install tree

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,241 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,978 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [ ]:
#@title Imports

import transformers
import evaluate

import nltk

import contractions

from datasets import load_dataset
from torchinfo import summary

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import os
import pandas as pd
import numpy as np

import scikit-learn

In [ ]:
# @title Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir_root = '/content/drive/MyDrive/266-final/'
# dir_data = '/content/drive/MyDrive/266-final/data/'
# dir_data = '/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/'
dir_data = '/content/drive/MyDrive/266-final/data/266-comp-lex-master'
dir_models = '/content/drive/MyDrive/266-final/models/'
dir_results = '/content/drive/MyDrive/266-final/results/'

In [ ]:
!tree /content/drive/MyDrive/266-final/data/266-comp-lex-master/

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/
├── evaluate.py
├── Readme.md
├── test
│   ├── lcp_multi_test.tsv
│   └── lcp_single_test.tsv
├── test-labels
│   ├── lcp_multi_test.tsv
│   └── lcp_single_test.tsv
├── train
│   ├── lcp_multi_train.tsv
│   └── lcp_single_train.tsv
└── trial
    ├── lcp_multi_trial.tsv
    └── lcp_single_trial.tsv

4 directories, 10 files


In [ ]:
!ls -R /content/drive/MyDrive/266-final/data/266-comp-lex-master/

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/:
evaluate.py  Readme.md	test  test-labels  train  trial

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/test:
lcp_multi_test.tsv  lcp_single_test.tsv

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/test-labels:
lcp_multi_test.tsv  lcp_single_test.tsv

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/train:
lcp_multi_train.tsv  lcp_single_train.tsv

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/trial:
lcp_multi_trial.tsv  lcp_single_trial.tsv


In [ ]:
#@title Import Data

In [ ]:
import pandas as pd
import os

# Set the base directory path
# dir_data = "/Users/home/VSCode/mids/266-fp/data/se21-t1-comp-lex-master"

# Load training datasets

train_single_df = pd.read_csv(os.path.join(dir_data, "train", "lcp_single_train.tsv"), sep="\t")
train_multi_df = pd.read_csv(os.path.join(dir_data, "train", "lcp_multi_train.tsv"), sep="\t")

trail_val_single_df = pd.read_csv(os.path.join(dir_data, "trial", "lcp_single_trial.tsv"), sep="\t")
trail_val_multi_df = pd.read_csv(os.path.join(dir_data, "trial", "lcp_multi_trial.tsv"), sep="\t")

test_single_df = pd.read_csv(os.path.join(dir_data, "test-labels", "lcp_single_test.tsv"), sep="\t")
test_multi_df = pd.read_csv(os.path.join(dir_data, "test-labels", "lcp_multi_test.tsv"), sep="\t")

# single_train_df = pd.read_csv(os.path.join(dir_data, "train", "lcp_single_train.tsv"), sep="\t")
# multi_train_df = pd.read_csv(os.path.join(dir_data, "train", "lcp_multi_train.tsv"), sep="\t")

# single_test_df = pd.read_csv(os.path.join(dir_data, "test", "lcp_single_test.tsv"), sep="\t")

# Try to load the problematic multi_test file with error handling
try:
    # Approach 1: Try with the C engine but with error handling
    multi_test_df = pd.read_csv(
        os.path.join(dir_data, "test", "lcp_multi_test.tsv"),
        sep="\t",
        on_bad_lines='skip'  # Skip bad lines
    )
    print("Loaded with skipping bad lines")
except Exception as e:
    print(f"First approach failed: {e}")
    try:
        # Approach 2: Try with the Python engine which might be more forgiving
        multi_test_df = pd.read_csv(
            os.path.join(dir_data, "test", "lcp_multi_test.tsv"),
            sep="\t",
            engine="python",
            quoting=3  # QUOTE_NONE
        )
        print("Loaded with Python engine")
    # except Exception as e:
    #     print(f"Second approach failed: {e}")
    #     # Approach 3: Let's try to manually identify and fix the problematic line
    #     with open(os.path.join(dir_data, "test", "lcp_multi_test.tsv"), 'r') as file:
    #         lines = file.readlines()

    #     print(f"Total lines in file: {len(lines)}")
    #     if len(lines) >= 40:
    #         print(f"Problematic line might be: {lines[39]}")  # Line 40 (0-indexed)

    #     # Create a dataframe from the lines we can read
    #     import io
    #     good_lines = lines[:39] + lines[40:] if len(lines) >= 40 else lines
    #     multi_test_df = pd.read_csv(io.StringIO(''.join(good_lines)), sep="\t")
    #     print("Loaded by skipping problematic line manually")

# # Display information about the loaded dataframes
# print(f"\nSingle word training data: {single_train_df.shape[0]} records with {single_train_df.shape[1]} columns")
# print(f"Multi word training data: {multi_train_df.shape[0]} records with {multi_train_df.shape[1]} columns")
# print(f"Single word test data: {single_test_df.shape[0]} records with {single_test_df.shape[1]} columns")
# print(f"Multi word test data: {multi_test_df.shape[0]} records with {multi_test_df.shape[1]} columns")

# # Load test labels using the Python engine to avoid potential parsing issues
# single_test_labels_df = pd.read_csv(
#     os.path.join(dir_data, "test-labels", "lcp_single_test.tsv"),
#     sep="\t",
#     engine="python",
#     quoting=3  # QUOTE_NONE
# )

# multi_test_labels_df = pd.read_csv(
#     os.path.join(dir_data, "test-labels", "lcp_multi_test.tsv"),
#     sep="\t",
#     engine="python",
#     quoting=3  # QUOTE_NONE
# )

# # Display information about the loaded test label dataframes
# print(f"Single word test labels: {single_test_labels_df.shape[0]} records with {single_test_labels_df.shape[1]} columns")
# print(f"Multi word test labels: {multi_test_labels_df.shape[0]} records with {multi_test_labels_df.shape[1]} columns")


First approach failed: Error tokenizing data. C error: EOF inside string starting at row 40
Loaded with Python engine

Single word training data: 7232 records with 5 columns
Multi word training data: 1464 records with 5 columns
Single word test data: 808 records with 4 columns
Multi word test data: 184 records with 4 columns
Single word test labels: 917 records with 5 columns
Multi word test labels: 184 records with 5 columns
